In [79]:
from importlib import reload  # Python 3.4+ only.
import src
reload(src.features.utils)
from src.features.utils import decimate_ptr

import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from pathlib import Path
import os
import re
import pandas as pd

from tqdm import tqdm

In [80]:
h5_key = 'decimated'

basedir = Path('..')
datadir = basedir / 'data' / 'raw'
decimate_dir = basedir / 'data' / 'interim' / 'decimated'

In [81]:
## Grab filenames
def get_files(level='(e|n)',
              subject='([0-9]{3})',
              task='(.+?)'):
    files = []
    query = f'^{level}_sub-'
    query += f'{subject}_ses-1_'
    query += f'task-{task}\.csv'
    for f in os.listdir(datadir):
        match = re.search(query, f)
        if match:
            files.append((f, match.groups()))
    
    return(files)

In [85]:
paths[:3]

[('e_sub-025_ses-1_task-restingstate.csv', ('025',)),
 ('e_sub-050_ses-1_task-restingstate.csv', ('050',)),
 ('e_sub-052_ses-1_task-restingstate.csv', ('052',))]

In [86]:
## Load Paths
tasks = ['restingstate', 'openmonitoring', 'compassion']
levels = ['e', 'n']

for task,level in tqdm(np.array(np.meshgrid(tasks,levels)).T.reshape(-1,2)):
    paths = get_files(level=level, task=task)
    
    ## Decimate and store again
    for path,subjs in paths:
        temp = decimate_ptr(pd.read_csv(datadir / path, header = None).to_numpy())
        subj = subjs[0]
        h5f = h5py.File(decimate_dir / f'{level}_sub-{subj}_ses-1_task-{task}_decimate.h5', 'w')
        h5f.create_dataset(h5_key, data=temp)
        h5f.close()
        temp = None

100%|██████████| 6/6 [14:33<00:00, 149.57s/it]
